In [1]:
import pandas as pd 
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import os, sys, time
import util
from model import Generator, Discriminator
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers
tf.keras.backend.set_floatx('float32')
from sklearn.impute import SimpleImputer

In [2]:
from numba import cuda
os.environ['CUDA_VISIBLE_DEVICES']="0"
print(tf.test.is_gpu_available())

True


In [3]:
df=pd.read_csv("./dataset/df_noOutliner_ana.csv",index_col=0)

In [4]:
#  ## numerical (17)
# num = ['BT_NM', 'HR_NM', 'RR_NM', 'HB_NM', 'HCT_NM', 'PLATELET_NM', 'WBC_NM', 'PTT1_NM',
#       'PTT2_NM', 'PTINR_NM', 'ER_NM', 'BUN_NM', 'CRE_NM', 'BMI', 'age', 'NIHTotal',
#       'PPD', ]
# ## category (55)
# cat = ['THD_ID', 'THDA_FL', 'THDH_FL', 'THDI_FL',
#           'THDAM_FL', 'THDV_FL', 'THDE_FL', 'THDM_FL', 'THDR_FL', 'THDP_FL',
#           'THDOO_FL', 'Gender', 'cortical_ACA_ctr', 'cortical_MCA_ctr', 'subcortical_ACA_ctr',
#           'subcortical_MCA_ctr', 'PCA_cortex_ctr', 'thalamus_ctr',
#           'brainstem_ctr', 'cerebellum_ctr', 'Watershed_ctr',
#           'Hemorrhagic_infarct_ctr', 'cortical_ACA_ctl', 'cortical_MCA_ctl',
#           'subcortical_ACA_ctl', 'subcortical_MCA_ctl', 'PCA_cortex_ctl',
#           'thalamus_ctl', 'brainstem_ctl', 'cerebellum_ctl', 'Watershed_ctl',
#           'Hemorrhagic_infarct_ctl', 'cortical_CT', 'subcortical_CT',
#           'circulation_CT', 'CT_find', 'watershed_CT', 'Hemorrhagic_infarct_CT',
#           'CT_left', 'CT_right', 'NIHS_1a_in', 'NIHS_1b_in', 'NIHS_1c_in',
#           'NIHS_2_in', 'NIHS_3_in', 'NIHS_4_in', 'NIHS_5aL_in', 'NIHS_5bR_in',
#           'NIHS_6aL_in', 'NIHS_6bR_in', 'NIHS_7_in', 'NIHS_8_in', 'NIHS_9_in',
#           'NIHS_10_in', 'NIHS_11_in', ]
# ## Label (1)
# label = df[['elapsed_class']]
# # imputation
# imp_mode = SimpleImputer(strategy='most_frequent')
# imp_mean = SimpleImputer(strategy='mean')
# df[cat] = imp_mode.fit_transform(df[cat])
# df[num] = imp_mean.fit_transform(df[num])
# df = pd.concat([df[num], df[cat], label], axis=1)

In [5]:
# df, imp_mode, imp_mean=util.FeatureArrange(df)
df

,BT_NM,HR_NM,RR_NM,HB_NM,HCT_NM,PLATELET_NM,WBC_NM,PTT1_NM,PTT2_NM,PTINR_NM,...,NIHS_5aL_in,NIHS_5bR_in,NIHS_6aL_in,NIHS_6bR_in,NIHS_7_in,NIHS_8_in,NIHS_9_in,NIHS_10_in,NIHS_11_in,elapsed_class
UUID,,,,,,,,,,,,,,,,,,,,,
014-00022421-1TSYM-007B12PW,36.4,73.0,16.0,12.0,40.934315,164.0,5.50,29.700000,26.0,0.94,...,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0
038-00089525-1TSYM-007TYKPH,36.6,72.0,18.0,13.7,40.500000,154.0,6.72,26.000000,28.6,1.02,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
023-00111356-1TSYM-00HJDR2J,36.8,103.0,22.0,14.3,37.900000,297.0,15.10,17.000000,34.6,1.55,...,0.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
038-00102625-1TSYM-00KWVJEC,37.1,89.0,20.0,9.7,29.300000,128.0,5.57,25.600000,28.7,0.97,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
002-00018873-1TSYM-011LJUYL,36.6,67.0,21.0,14.0,40.934315,269.0,8.70,29.300000,26.0,0.97,...,2.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
048-00108913-1TSYM-ZYTDM73A,37.0,68.0,20.0,14.0,41.900000,326.0,8.00,31.900000,30.6,0.95,...,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
009-00062920-1TSYM-ZYZF0KMA,36.2,65.0,17.0,12.1,35.400000,163.0,5.80,30.000000,32.0,0.94,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
034-00039440-1TSYM-ZZ1WFW3K,37.5,72.0,20.0,13.3,39.300000,267.0,6.80,28.500000,28.3,0.93,...,4.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [ ]:
# df.isnull().sum()

In [ ]:
sc = MinMaxScaler()
df = sc.fit_transform(df)
df.shape

In [ ]:
## setting hyperparameter
latent_dim = 73
epochs = 1
batch_size= 128
buffer_size = 6000
# save_interval = 50
n_critic = 5

In [ ]:
generator = Generator()
discriminator = Discriminator()

In [ ]:
gen_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)
disc_opt = tf.keras.optimizers.Adam(0.0001, 0.5, 0.9)

In [ ]:
training_dataset=tf.data.Dataset.from_tensor_slices(df.astype('float32'))\
    .shuffle(buffer_size).batch(batch_size, drop_remainder=True)

In [ ]:
@tf.function
def train_step(x):
    noise = tf.random.normal([batch_size, latent_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as dis_tape:
        gen_data = generator(noise, training=True)
        gen_output = discriminator(gen_data, training=True)
        real_output = discriminator(x, training=True )
        
#         print("gen_output:",gen_output)
#         print("real_output:",real_output)
        
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat,)
#         print("x_hat:",x_hat)
#         print("d_hat:",d_hat)
        
        disc_loss = util.discriminator_loss(real_output, gen_output, d_hat, x_hat)
#         print("disc_loss:",disc_loss)
        gen_loss = util.generator_loss(gen_output)
#         print("gen_loss:",gen_loss)
        
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))
    return gen_output, real_output,x_hat,d_hat

In [ ]:
@tf.function
def train_discriminator(x):
    
    noise = tf.random.normal([x.shape[0], latent_dim])

    with tf.GradientTape() as dis_tape:
        gen_data = generator(noise, training=True)
#         print("gen_data",gen_data)
        gen_output = discriminator(gen_data,training=True)
#         print("gen_output",gen_output)
        real_output = discriminator(x, training=True)
#         print("real_output",real_output)
        x_hat = util.random_weight_average(x, gen_data)
        d_hat = discriminator(x_hat,)
        
        disc_loss = util.discriminator_loss(real_output, gen_output, d_hat, x_hat)
    grad_disc = dis_tape.gradient(disc_loss, discriminator.trainable_variables)
    disc_opt.apply_gradients(zip(grad_disc, discriminator.trainable_variables))

    return disc_loss


@tf.function
def train_generator():
    noise = tf.random.normal([batch_size, latent_dim])

    with tf.GradientTape() as gen_tape:
        gen_data = generator(noise, training=True )
        gen_output = discriminator(gen_data,training=True)

        gen_loss = util.generator_loss(gen_output)

    grad_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gen_opt.apply_gradients(zip(grad_gen, generator.trainable_variables))

    return gen_loss

In [ ]:
for epoch in range(epochs):
    start = time.time()
    disc_loss = 0
    gen_loss = 0
    
    for data in training_dataset:
#         gen_output, real_output,x_hat,d_hat=train_step(data)
        disc_loss += train_discriminator(data)
#         print("gen_output:",gen_output)
#         print("real_output", real_output,)
#         print("x_hat", x_hat)
#         print("d_hat",d_hat)
    
        if disc_opt.iterations.numpy() % n_critic == 0:
            gen_loss += train_generator()
    print('Time for epoch {} is {} sec - gen_loss = {}, disc_loss = {}'.format(epoch + 1,
                                                                               time.time() - start, gen_loss / batch_size, disc_loss / (batch_size*n_critic)))

In [ ]:
# disc_opt.iterations.numpy()

In [ ]:
print(12)